# Section 7 – Game Loop & Updates (dt best practices)

In Arcade, your “game loop” lives in two core callbacks:

- `on_update(self, dt)`: **update state** (movement, timers, physics).
- `on_draw(self)`: **render** the current state (no heavy logic here).

Key ideas:
- Use **pixels per second** for speeds, and multiply by **`dt`** to get frame-rate independence.
- Keep input -> **state** (velocity, targets) -> **update with dt** -> **draw**.
- For timed events/animation, use a **time accumulator**: `acc += dt`; when `acc >= period`, trigger and subtract.

We’ll demo:
1. **Time-based motion + simple animation** using a texture flip timer.
2. **Bouncing + gravity** using velocity and acceleration with `dt`.

---

# Exercise 7.1 – Timers & dt
- Change the flip rate to 12 FPS by adjusting `frame_period`.
- Make the coin move **left** and wrap on the **left** edge.
- Pause/resume animation with a key (e.g., `P`) by skipping the animation code when paused.


In [ ]:
import arcade, math

SCREEN_WIDTH, SCREEN_HEIGHT = 800, 600
TITLE = "Robot animation"

class DtBasics(arcade.Window):
    def __init__(self):
        super().__init__(SCREEN_WIDTH, SCREEN_HEIGHT, TITLE)
        arcade.set_background_color(arcade.color.ASH_GREY)

        self.robot_list = arcade.SpriteList()
        self.robot = arcade.Sprite(":resources:images/animated_characters/robot/robot_walk0.png", scale=1.0)
        self.robot.center_x, self.robot.center_y = 80, SCREEN_HEIGHT // 2
        self.robot_list.append(self.robot)

        self.speed_x = 180.0

        self.frames = [
            arcade.load_texture(":resources:images/animated_characters/robot/robot_walk0.png"),
            arcade.load_texture(":resources:images/animated_characters/robot/robot_walk1.png"),
            arcade.load_texture(":resources:images/animated_characters/robot/robot_walk2.png"),
            arcade.load_texture(":resources:images/animated_characters/robot/robot_walk3.png"),
        ]
        self.frame_index = 0
        self.frame_period = 0.125
        self.frame_acc = 0.0

        # wobble settings (no drift)
        self.t = 0.0
        self.wobble_amplitude_deg = 4.0 # +/- degrees
        self.wobble_speed_hz = 1.5 # cycles per second

    def on_draw(self):
        self.clear()
        self.robot_list.draw()
        arcade.draw_text(
            "Robot walks right at 180 px/s • 8 FPS walk cycle",
            16, 16, arcade.color.BLACK, 16
        )

    def on_update(self, dt):
        # Move
        self.robot.center_x += self.speed_x * dt
        if self.robot.left > SCREEN_WIDTH:
            self.robot.right = 0
        if self.robot.right < 0:
            self.robot.left = SCREEN_WIDTH

        # Animate frames
        self.frame_acc += dt
        if self.frame_acc >= self.frame_period:
            self.frame_acc -= self.frame_period
            self.frame_index = (self.frame_index + 1) % len(self.frames)
            self.robot.texture = self.frames[self.frame_index]

        # Wobble 
        self.t += dt
        self.robot.angle = self.wobble_amplitude_deg * math.sin(2 * math.pi * self.wobble_speed_hz * self.t)

window = DtBasics()
arcade.run()


# Section 7 – Game Loop & Updates (dt best practices)

In Arcade, your “game loop” lives in two core callbacks:

- `on_update(self, dt)`: **update state** (movement, timers, physics).
- `on_draw(self)`: **render** the current state (no heavy logic here).

Key ideas:
- Use **pixels per second** for speeds, and multiply by **`dt`** to get frame-rate independence.
- Keep input -> **state** (velocity, targets) -> **update with dt** -> **draw**.
- For timed events/animation, use a **time accumulator**: `acc += dt`; when `acc >= period`, trigger and subtract.

We’ll demo:
1. **Time-based motion + simple animation** using a texture flip timer.
2. **Bouncing + gravity** using velocity and acceleration with `dt`.

In [ ]:
import arcade
import random
import math

SCREEN_WIDTH, SCREEN_HEIGHT = 1000, 1000
TITLE = "Bouncing Coins + Gems"

BALL_COUNT = 10
GRAVITY = -900
REST = 0.85

# Explostion tuning
EXPLOSION_RADIUS =  180
EXPLOSION_STRENGTH = 10000

TEXTURE_PATHS = [
    ":resources:images/items/coinGold.png",
    ":resources:images/items/gemBlue.png",
    ":resources:images/items/gemGreen.png",
    ":resources:images/items/gemRed.png",
]

class Ball(arcade.Sprite):
    def __init__(self, x, y):
        # pick a random texture
        texture_path = random.choice(TEXTURE_PATHS)
        super().__init__(texture_path, scale=0.6)
        self.center_x = x
        self.center_y = y
        self.vx = random.uniform(-250, 250)
        self.vy = random.uniform(-100, 200)
        self.mass = 1.0

class DarkerShadow(arcade.Window):
    def __init__(self):
        super().__init__(SCREEN_WIDTH, SCREEN_HEIGHT, TITLE)
        arcade.set_background_color(arcade.color.SKY_BLUE)

        self.ball_list = arcade.SpriteList()
        for _ in range(BALL_COUNT):
            x = random.randint(100, SCREEN_WIDTH - 100)
            y = random.randint(300, SCREEN_HEIGHT - 50)
            self.ball_list.append(Ball(x, y))

        self.ground_h = 60
        self.ground_rect = arcade.rect.LBWH(0, 0, SCREEN_WIDTH, self.ground_h)

        self.trails = []
        self.explosions = []

    def on_draw(self):
        self.clear()
        arcade.draw_rect_filled(self.ground_rect, arcade.color.DARK_SPRING_GREEN)

        # Trails
        for x, y, alpha in self.trails:
            arcade.draw_circle_filled(x, y, 7, (255, 255, 255, int(alpha)))

        # Explosion rings
        for x, y, radius, alpha in self.explosions:
            arcade.draw_circle_outline(x, y, radius, (255, 255, 255, int(alpha)), border_width=3)

        self.ball_list.draw()

    def physics_step(self, dt):
        for ball in self.ball_list:
            ball.vy += GRAVITY * dt

        # Move
        ball.center_x += ball.vx * dt
        ball.center_y += ball.vy * dt

        # Floor bounce
        floor_y = self.ground_h + ball.height / 2
